In [5]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter
import numpy as np
import itertools
import copy
import matplotlib.pyplot as plt
import json
import time
from dataraw_sampling import sample_queries, sample_queries_simple_shatter, resolve_eval_expression, decode_key_to_vocab_token

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
num_attributes=2; num_attr_vals=3; query_length_multiplier=1; nest_depth_int=1; multiple_OR_sets_bool=True
N_train=0; N_val=256; N_test=256

expression_resolve_fn = resolve_eval_expression

sample_start_time = time.time()
data, stats = sample_queries_simple_shatter(
    num_attributes, query_length_multiplier, nest_depth_int, multiple_OR_sets_bool, 
    expression_resolve_fn,
    N_train, N_val, N_test,
    validate=False,
    debug=False)
sample_end_time = time.time()

print(f'Sample Time for {N_train+N_val+N_test} datapoints = {sample_end_time-sample_start_time} seconds')


save_start_time = time.time()
filename = '../Raw_Datasets/SET/SimpleShatter-{}Attr-{}Vals-{}LenMultiplier-{}NestDepth-{}Train-{}Val-{}Test.json'.format(
    num_attributes, num_attr_vals, query_length_multiplier, nest_depth_int, N_train, N_val, N_test)
with open(filename, 'w') as f:
    json.dump(data, f)
save_end_time = time.time()

filename = '../Raw_Datasets/SET/SimpleShatter-{}Attr-{}Vals-{}LenMultiplier-{}NestDepth-{}Train-{}Val-{}Test-Stats.json'.format(
    num_attributes, num_attr_vals, query_length_multiplier, nest_depth_int, N_train, N_val, N_test)
with open(filename, 'w') as f:
    json.dump(stats, f)
save_end_time = time.time()

print(f'Saving Time for {N_train+N_val+N_test} datapoints = {save_end_time-save_start_time} seconds')

100%|██████████| 512/512 [00:00<00:00, 25652.93it/s]

Sample Time for 512 datapoints = 0.02237224578857422 seconds
Saving Time for 512 datapoints = 0.005433559417724609 seconds


In [15]:
import operator as op
from functools import reduce

def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom


def derive_shatter_bucket_probs(num_cards):
    # (81 choose 1)*1, (81 choose 2)*2, .... (81 choose 81)*81
    num_datapoints_by_bucket = np.array([ncr(num_cards, subset_size) for subset_size in range(1, num_cards+1)])
    bucket_probs = num_datapoints_by_bucket / np.sum(num_datapoints_by_bucket)
    return bucket_probs.astype(float)

In [16]:
derive_shatter_bucket_probs(9)

array([0.01761252, 0.0704501 , 0.16438356, 0.24657534, 0.24657534,
       0.16438356, 0.0704501 , 0.01761252, 0.00195695])

In [7]:
data['val_tokens']

[([6, 4, 2, 5, 8, 3, 7, 1], 8),
 ([6, 8, 0, 4], 6),
 ([2, 8, 1, 0, 6], 0),
 ([6, 8, 5, 1, 2], 5),
 ([3, 4, 2, 0, 8], 3),
 ([6, 7, 4, 2], 4),
 ([6, 3, 2], 2),
 ([5, 0, 3, 2], 3),
 ([7, 8, 1, 5, 4, 0], 0),
 ([2, 7, 3, 4, 1], 3),
 ([5, 6, 2, 1, 8, 7], 6),
 ([8, 5, 7, 4], 8),
 ([2, 3, 1, 8, 5], 1),
 ([6, 0, 7, 8], 6),
 ([2, 7, 0, 3, 6, 5], 5),
 ([0, 6], 0),
 ([1, 7, 5, 4], 5),
 ([7, 8, 5, 2], 7),
 ([1, 2, 5, 0], 2),
 ([8, 4, 6, 3, 1], 3),
 ([8, 7, 5, 3, 0], 8),
 ([8, 1, 3, 0], 0),
 ([0, 7, 2, 4], 7),
 ([5, 2, 0], 0),
 ([3, 8, 6, 2, 1], 2),
 ([1, 8, 4, 3, 5], 4),
 ([3, 4, 7, 1, 8, 0, 6], 6),
 ([4, 3, 5, 7], 3),
 ([5, 6, 7, 3, 1], 3),
 ([5, 6, 7, 4], 5),
 ([7, 5, 4, 8, 0], 5),
 ([0, 6, 1], 1),
 ([2, 3, 4, 6, 0, 8], 4),
 ([5], 5),
 ([7, 3, 1, 4], 7),
 ([4, 8, 3, 7, 1], 3),
 ([8, 3, 7, 2, 1, 4, 6], 4),
 ([3, 4, 7, 0, 1], 1),
 ([1, 8, 5], 5),
 ([3, 8], 3),
 ([2, 0, 5, 6], 2),
 ([6, 0, 4, 7], 0),
 ([2, 3, 6], 3),
 ([0, 5, 4], 0),
 ([8, 2, 7, 5, 0], 8),
 ([2, 3, 6, 5, 8, 0, 1, 4], 8),
 ([7, 3, 8,

In [ ]:
# nest_depth = 8

defaultdict(<class 'float'>, {'nest_flat_query_parts': 0.15799379348754883, 'derive_missing_query_part': 1.0309646129608154, 'substitude_DERIVE_into_input_expression': 0.03975033760070801, 'convert_input_expression_to_vocab_tokens': 1.7907702922821045, 'Post set up sampling': 3.3711490631103516})
Sample Time for 512 datapoints = 8.825998544692993 seconds
Saving Time for 512 datapoints = 1.3774933815002441 seconds

In [ ]:
# nest_depth = 1

defaultdict(<class 'float'>, {'nest_flat_query_parts': 0.45519256591796875, 'derive_missing_query_part': 12.749177694320679, 'substitude_DERIVE_into_input_expression': 0.0369265079498291, 'convert_input_expression_to_vocab_tokens': 1.7782859802246094, 'Post set up sampling': 15.444261074066162})
Sample Time for 512 datapoints = 20.80058002471924 seconds
Saving Time for 512 datapoints = 1.2895803451538086 seconds

In [ ]:
# 1. sample on the fly?

# 1. how long does it take to sample 512 on the fly?

In [23]:
# problem -- cannot fit in memory

16e6 / 30720 * 608 / 1000

316.6666666666667

In [29]:
16e6 / 30720 * 274.0126442909241 / 3600

39.643033028200826

In [ ]:
# data failure mode:
# 1. it doesn't do well on larger buckets, or smaller buckets -- Not much training data. But training objective doesn't directly as model to predict how many gt there are.
# 2. it doesn't do well on more nested queries -- transformer is not deep enough?
# 3. it doesn't do well on longer queries -- transformer is not deep enough?

In [5]:
stats.keys()

dict_keys(['bucket_counts', 'or_op_counts', 'and_op_counts', 'depth_counts', 'input_lens'])

In [3]:
sorted(stats['input_lens'].items(), key=lambda x:x[1])

[(29, 16),
 (33, 16),
 (37, 18),
 (31, 21),
 (35, 21),
 (19, 47),
 (27, 225),
 (25, 758),
 (23, 2010),
 (21, 7108)]

In [12]:
data['train_tokens'][0][1]

[1, 5, 6, 11, 13, 17]

In [34]:
sorted(stats['bucket_counts'].items(), key=lambda x:x[1])

[(9, 42),
 (1, 524),
 (8, 528),
 (2, 2124),
 (7, 2160),
 (3, 5034),
 (6, 5094),
 (5, 7575),
 (4, 7639)]

In [35]:
sorted(stats['or_op_counts'].items(), key=lambda x:x[1])

[(269, 1),
 (247, 1),
 (584, 1),
 (640, 2),
 (608, 2),
 (221, 2),
 (313, 3),
 (227, 3),
 (553, 3),
 (289, 3),
 (413, 3),
 (253, 3),
 (359, 3),
 (131, 3),
 (271, 3),
 (47, 4),
 (97, 4),
 (353, 4),
 (209, 4),
 (427, 4),
 (13, 4),
 (137, 4),
 (89, 4),
 (317, 4),
 (71, 4),
 (107, 4),
 (581, 4),
 (293, 4),
 (371, 4),
 (19, 4),
 (59, 4),
 (83, 4),
 (637, 4),
 (37, 4),
 (197, 5),
 (127, 5),
 (41, 5),
 (193, 5),
 (281, 5),
 (497, 5),
 (319, 5),
 (113, 5),
 (263, 5),
 (539, 5),
 (299, 5),
 (181, 5),
 (337, 5),
 (157, 5),
 (163, 5),
 (602, 5),
 (101, 5),
 (241, 6),
 (311, 6),
 (616, 6),
 (323, 6),
 (169, 6),
 (29, 6),
 (223, 6),
 (191, 6),
 (567, 6),
 (199, 6),
 (301, 6),
 (1, 6),
 (349, 6),
 (574, 6),
 (283, 6),
 (139, 6),
 (17, 6),
 (251, 7),
 (233, 7),
 (11, 7),
 (518, 7),
 (341, 7),
 (103, 7),
 (187, 7),
 (151, 7),
 (469, 7),
 (239, 7),
 (229, 7),
 (79, 7),
 (31, 7),
 (277, 7),
 (511, 7),
 (23, 7),
 (331, 7),
 (609, 7),
 (307, 8),
 (67, 8),
 (43, 8),
 (121, 8),
 (149, 8),
 (109, 8),
 (203, 9

In [36]:
sorted(stats['and_op_counts'].items(), key=lambda x:x[1])

[(459, 1),
 (481, 1),
 (144, 1),
 (88, 2),
 (120, 2),
 (507, 2),
 (415, 3),
 (501, 3),
 (175, 3),
 (439, 3),
 (315, 3),
 (475, 3),
 (369, 3),
 (597, 3),
 (457, 3),
 (681, 4),
 (631, 4),
 (375, 4),
 (519, 4),
 (301, 4),
 (715, 4),
 (591, 4),
 (639, 4),
 (411, 4),
 (657, 4),
 (621, 4),
 (147, 4),
 (435, 4),
 (357, 4),
 (709, 4),
 (669, 4),
 (645, 4),
 (91, 4),
 (691, 4),
 (531, 5),
 (601, 5),
 (687, 5),
 (535, 5),
 (447, 5),
 (231, 5),
 (409, 5),
 (615, 5),
 (465, 5),
 (189, 5),
 (429, 5),
 (547, 5),
 (391, 5),
 (571, 5),
 (565, 5),
 (126, 5),
 (627, 5),
 (487, 6),
 (417, 6),
 (112, 6),
 (405, 6),
 (559, 6),
 (699, 6),
 (505, 6),
 (537, 6),
 (161, 6),
 (529, 6),
 (427, 6),
 (727, 6),
 (379, 6),
 (154, 6),
 (445, 6),
 (589, 6),
 (711, 6),
 (477, 7),
 (495, 7),
 (717, 7),
 (210, 7),
 (387, 7),
 (625, 7),
 (541, 7),
 (577, 7),
 (259, 7),
 (489, 7),
 (499, 7),
 (649, 7),
 (697, 7),
 (451, 7),
 (217, 7),
 (705, 7),
 (397, 7),
 (119, 7),
 (421, 8),
 (661, 8),
 (685, 8),
 (607, 8),
 (579, 8),
 

In [ ]:
# sample nest length = 1; query_length = 3 ** 6; multiple_OR_sets_bool=True

# num_attributes = 2; query_length_multiplier = 3 ** 4
# num_attributes = 3; query_length_multiplier = 3 ** 3
# num_attributes = 4; query_length_multiplier = 3 ** 2
# num_attributes = 5; query_length_multiplier = 3 ** 1
# num_attributes = 6; query_length_multiplier = 3 ** 6

In [3]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=1; nest_depth_int=1; multiple_OR_sets_bool=True
# N_train=2048; N_val=2048; N_test=2048

16e6 / 3072 * 5.024206638336182 / 3600

7.268817474444707

In [2]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=9; nest_depth_int=1; multiple_OR_sets_bool=True
# N_train=1024; N_val=1024; N_test=1024

16e6 / 3072 * 31.317060947418213 / 3600

45.30824789846385

In [6]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=3; nest_depth_int=1; multiple_OR_sets_bool=True
# N_train=1024; N_val=1024; N_test=1024

16e6 / 3072 * 8.630649089813232 / 3600

12.486471484104792

In [8]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=1; multiple_OR_sets_bool=True
# N_train=1024; N_val=1024; N_test=1024

16e6 / 3072 * 5.308434247970581 / 3600

7.680026400420401

In [11]:
# num_attributes=4; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=4; multiple_OR_sets_bool=True
# N_train=1024; N_val=1024; N_test=1024

16e6 / 3072 * 3.8778438568115234 / 3600

5.610306505803708

In [15]:
# num_attributes=6; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=4; multiple_OR_sets_bool=True
# N_train=2048; N_val=2048; N_test=2048

16e6 / 3072 * 138.9612045288086 / 3600

201.04340932987353

In [ ]:
# num_attributes=6; num_attr_vals=3; query_length_multiplier=2; nest_depth_int=4; multiple_OR_sets_bool=True
# N_train=2048; N_val=2048; N_test=2048

16e6 / 3072 * 7.696515083312988 / 3600

In [ ]:
# num_attributes=4; num_attr_vals=3; nest_depth = 4; num_cards multiple 2
16000000 / 1000 * 1.32326340675354 / 3600

In [28]:
import torch
a = torch.tensor([2]).cuda()
a

tensor([2], device='cuda:0')

In [30]:
a.detach().cpu()

tensor([2])